## TYTANチュートリアル（お絵かきロジック）

2023年4月27日

ビネクラ安田

出典：[量子アニーリング（D-wave）でお絵かきロジック（ロジックアート、ピクロス）を解く](https://vigne-cla.com/21-11/)

### 問題
QUBOでこの問題を解く。いかにも「n個の量子ビットからm個を1にする」と相性が良さそう。各マスに量子ビットを割り当てる。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/04/21-11_1.png" width = 40%>
</div>

### QUBOモデルでは何が設定できるか？（おさらい）

**<font color="red">「n個の量子ビットからm個を1にする」</font>**

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

**<font color="red">「2個の量子ビットが同時に1になったら報酬（またはペナルティ）を与える」</font>**

例）2個の量子ビットが同時に1になったらペナルティ0.5
```
H = 0.5 * (q1 * q2)
```

今回はこれらだけを使うが、その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件とコスト

２以上の単発の数字の場合は、「ｎ個の量子ビットからｍ個を１にする」を使って合計数を指定し、さらに、「2個の量子ビットが同時に１になったら報酬を与える」を使って**できるだけ隣り合うマスが同時に１になるようにする。**

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/04/21-11_2-1024x191.png" width = 70%>
</div>



１が並ぶスプリットは、同様に合計数を指定し、こちらは「2個の量子ビットが同時に１になったらペナルティを与える」を使って**できるだけ隣り合うマスが同時に１にならないようにする。**

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/04/21-11_3-1024x191.png" width = 70%>
</div>


なお、[1, 2] のようなスプリットは設定できない。


## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [5]:
from sympy import Symbol
from tytan import qubo, sampler
import numpy as np

#量子ビットを用意する
q00 = Symbol('q00')
q01 = Symbol('q01')
q02 = Symbol('q02')
q03 = Symbol('q03')
q04 = Symbol('q04')
q05 = Symbol('q05')
q06 = Symbol('q06')
q07 = Symbol('q07')
q08 = Symbol('q08')
q09 = Symbol('q09')
q10 = Symbol('q10')
q11 = Symbol('q11')
q12 = Symbol('q12')
q13 = Symbol('q13')
q14 = Symbol('q14')
q15 = Symbol('q15')
q16 = Symbol('q16')
q17 = Symbol('q17')
q18 = Symbol('q18')
q19 = Symbol('q19')
q20 = Symbol('q20')
q21 = Symbol('q21')
q22 = Symbol('q22')
q23 = Symbol('q23')
q24 = Symbol('q24')

#縦方向の個数の制約
H = 0
H += (q00 + q05 + q10 + q15 + q20 - 3)**2
H += (q01 + q06 + q11 + q16 + q21 - 2)**2
H += (q02 + q07 + q12 + q17 + q22 - 5)**2
H += (q03 + q08 + q13 + q18 + q23 - 2)**2
H += (q04 + q09 + q14 + q19 + q24 - 1)**2

#縦方向の連続の報酬
H += -0.1 * (q00 * q05) -0.1 * (q05 * q10) -0.1 * (q10 * q15) -0.1 * (q15 * q20)
H += -0.1 * (q01 * q06) -0.1 * (q06 * q11) -0.1 * (q11 * q16) -0.1 * (q16 * q21)
H += -0.1 * (q02 * q07) -0.1 * (q07 * q12) -0.1 * (q12 * q17) -0.1 * (q17 * q22)
H += -0.1 * (q03 * q08) -0.1 * (q08 * q13) -0.1 * (q13 * q18) -0.1 * (q18 * q23)
#1個の列は連続設定なし

#横方向の個数の制約
H += (q00 + q01 + q02 + q03 + q04 - 2)**2
H += (q05 + q06 + q07 + q08 + q09 - 3)**2
H += (q10 + q11 + q12 + q13 + q14 - 3)**2
H += (q15 + q16 + q17 + q18 + q19 - 3)**2
H += (q20 + q21 + q22 + q23 + q24 - 2)**2

#横方向の連続の報酬
H += -0.1 * (q00 * q01) -0.1 * (q01 * q02) -0.1 * (q02 * q03) -0.1 * (q03 * q04)
H += -0.1 * (q05 * q06) -0.1 * (q06 * q07) -0.1 * (q07 * q08) -0.1 * (q08 * q09)
H += -0.1 * (q10 * q11) -0.1 * (q11 * q12) -0.1 * (q12 * q13) -0.1 * (q13 * q14)
H += -0.1 * (q15 * q16) -0.1 * (q16 * q17) -0.1 * (q17 * q18) -0.1 * (q18 * q19)
#[1, 1]の行は別で設定する

#横方向の[1, 1]スプリットのペナルティ
H += 0.1 * (q20 * q21) + 0.1 * (q21 * q22) + 0.1 * (q22 * q23) + 0.1 * (q23 * q24)


#コンパイル
QUBO, offset = qubo.Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(QUBO, shots=500)

#上位5件
for r in result[:5]:
    print(r)

#上位2件
print(np.array(list(result[0][0].values()), int).reshape(5, 5))
print(np.array(list(result[1][0].values()), int).reshape(5, 5))

[{'q00': 0.0, 'q01': 0.0, 'q02': 1.0, 'q03': 1.0, 'q04': 0.0, 'q05': 0.0, 'q06': 0.0, 'q07': 1.0, 'q08': 1.0, 'q09': 1.0, 'q10': 1.0, 'q11': 1.0, 'q12': 1.0, 'q13': 0.0, 'q14': 0.0, 'q15': 1.0, 'q16': 1.0, 'q17': 1.0, 'q18': 0.0, 'q19': 0.0, 'q20': 1.0, 'q21': 0.0, 'q22': 1.0, 'q23': 0.0, 'q24': 0.0}, -79.5, 53]
[{'q00': 0.0, 'q01': 0.0, 'q02': 1.0, 'q03': 1.0, 'q04': 0.0, 'q05': 1.0, 'q06': 0.0, 'q07': 1.0, 'q08': 1.0, 'q09': 0.0, 'q10': 1.0, 'q11': 1.0, 'q12': 1.0, 'q13': 0.0, 'q14': 0.0, 'q15': 1.0, 'q16': 1.0, 'q17': 1.0, 'q18': 0.0, 'q19': 0.0, 'q20': 0.0, 'q21': 0.0, 'q22': 1.0, 'q23': 0.0, 'q24': 1.0}, -79.4, 24]
[{'q00': 0.0, 'q01': 1.0, 'q02': 1.0, 'q03': 0.0, 'q04': 0.0, 'q05': 0.0, 'q06': 0.0, 'q07': 1.0, 'q08': 1.0, 'q09': 1.0, 'q10': 1.0, 'q11': 0.0, 'q12': 1.0, 'q13': 1.0, 'q14': 0.0, 'q15': 1.0, 'q16': 1.0, 'q17': 1.0, 'q18': 0.0, 'q19': 0.0, 'q20': 1.0, 'q21': 0.0, 'q22': 1.0, 'q23': 0.0, 'q24': 0.0}, -79.3, 24]
[{'q00': 0.0, 'q01': 1.0, 'q02': 1.0, 'q03': 0.0, 'q04': 0